<a href="https://colab.research.google.com/github/duvancatan/Time_Series_UdeA/blob/main/3_AMAZON_Aplicaciones_ARIMA_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#PRONÓSTICO DE VENTAS DE AMAZON
Los siguientes datos representan las ventas promedio mensual de un artefacto electrónico de Amazon. Se tienen 150 datos disponibles. El gerente de ventas está interesado en conocer cuánto serán las ventas mensuales en el próximo año dada la información disponible.

## Instalación y carga de librerías  

In [ ]:
# Instalación de la librería statsmodels #
!pip install statsmodels

In [ ]:
# Carga de librerías #
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_absolute_error, mean_squared_error

## Lectura de datos

In [ ]:
# Carga de datos #
sales =  (10.01 ,10.07	,10.32	,9.75	,10.33 ,10.13	,10.36	,10.32	,10.13	,10.16	,10.58	,10.62	,10.86	,11.2,
          10.74 ,10.56	,10.48	,10.77	,11.33	,10.96	,11.16	,11.7	,11.39	,11.42	,11.94	,11.24	,11.59	,10.96,
          11.4	,11.02	,11.01	,11.23	,11.33	,10.83	,10.84	,11.14	,10.38	,10.9	,11.05	,11.11	,11.01	,11.22,
          11.21 ,11.91	,11.69	,10.93	,10.99	,11.01	,10.84	,10.76	,10.77	,10.88	,10.49	,10.5	,11	,10.98,
          10.61 ,10.48	,10.53	,11.07	,10.61	,10.86	,10.34	,10.78	,10.8	,10.33	,10.44	,10.5	,10.75	,10.4	,10.4,
          10.34 ,10.55	,10.46	,10.82	,10.91	,10.87	,10.67	,11.11	,10.88	,11.28	,11.27	,11.44	,11.52	,12.1,
          11.83 ,12.62, 12.41	,12.43	,12.73	,13.01	,12.74	,12.73	,12.76	,12.92	,12.64	,12.79	,13.05	,12.69,
          13.01 ,12.9	,13.12, 12.47	,12.47 ,12.94	,13.1	,12.91 ,13.39	,13.13	,13.34	,13.34	,13.14	,13.49	,13.87,
          13.39 ,13.59	,13.27, 13.7	,13.2	,13.32	,13.15	,13.3	,12.94	,13.29	,13.26	,13.08	,13.24	,13.31	,13.52,
          13.02 ,13.25	,13.12	,13.26, 13.11	,13.3	,13.06	,13.32	,13.1	,13.27	,13.64	,13.58	,13.87	,13.53	,13.41,
          13.25 ,13.5	,13.58	,13.51	,13.77	,13.4	)

## Formato y Visualización

In [ ]:
# Formato y rango temporal #
date_range = pd.date_range(start='2011-12-01', periods=150, freq='M')
series = pd.Series(sales, index=date_range)

In [ ]:
# Visualizar la serie de tiempo #
plt.figure(figsize=(8, 4))
plt.plot(series)
plt.title('Serie de tiempo de Ventas de Amazon')
plt.show()

## Tranformaciones y Gráficos

In [ ]:
# Diferenciar la serie #
diff_series = series.diff().dropna()

In [ ]:
# Visualizar la serie diferenciada #
plt.figure(figsize=(8, 4))
plt.plot(diff_series)
plt.title('Serie de Ventas Diferenciada')
plt.show()

## Ajuste y Estimación

In [ ]:
# Ajustar el modelo ARIMA(0,1,1) #
model = ARIMA(series, order=(0, 1, 1))
model_fit = model.fit()

In [ ]:
# Resumen del modelo ajustado #
print(model_fit.summary())

## Pronósticos

In [ ]:
# Realizar predicciones con intervalos de confianza #
forecast_steps = 24
forecast = model_fit.get_forecast(steps=forecast_steps)
forecast_mean = forecast.predicted_mean
forecast_conf_int = forecast.conf_int()

In [ ]:
print(forecast_mean)

In [ ]:
# Visualizar predicciones junto con la serie original y los intervalos de predicción #
plt.figure(figsize=(8, 4))
plt.plot(series, label='Serie original')
plt.plot(pd.date_range(start=series.index[-1], periods=forecast_steps, freq='D'), forecast_mean, label='Predicción', color='red')
plt.fill_between(pd.date_range(start=series.index[-1], periods=forecast_steps, freq='D'),
                 forecast_conf_int.iloc[:, 0], forecast_conf_int.iloc[:, 1], color='pink', alpha=0.3)
plt.title('Predicción con ARIMA(0,1,1) e intervalos de confianza')
plt.legend()
plt.show()

## Evaluación de Pronósticos

In [ ]:
# Función para calcular el MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Generar predicciones en el conjunto de entrenamiento
train_predictions = model_fit.predict(start=1, end=len(series)-1, dynamic=False)
train_truth = series[1:]

# Calcular el MAE, RMSE y MAPE
mae = mean_absolute_error(train_truth, train_predictions)
rmse = np.sqrt(mean_squared_error(train_truth, train_predictions))
mape = mean_absolute_percentage_error(train_truth, train_predictions)

print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'MAPE: {mape:.2f}%')